# Load and Inspect Data

In this section, we will load the CSV file into a Pandas DataFrame, inspect the first few rows, and check for missing values.

In [2]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('power_system_data.csv')

# Inspect the first few rows
print(data.head())

# Check for missing values
print(data.isnull().sum())

             timestamp station_id  voltage_pu  current_pu  real_power_mw  \
0  2024-01-01 00:00:00    SUB_001    0.997235   69.039656      68.848741   
1  2024-01-01 01:00:00    SUB_001    1.006378   63.150897      63.553676   
2  2024-01-01 02:00:00    SUB_001    1.015349   78.702437      79.910417   
3  2024-01-01 03:00:00    SUB_001    0.989497   72.880458      72.114965   
4  2024-01-01 04:00:00    SUB_001    0.961734   82.441143      79.286483   

   reactive_power_mvar  power_factor  
0            31.408689      0.909799  
1            43.799332      0.823399  
2            59.290527      0.803088  
3            48.928608      0.827511  
4            46.036720      0.864792  
timestamp              0
station_id             0
voltage_pu             0
current_pu             0
real_power_mw          0
reactive_power_mvar    0
power_factor           0
dtype: int64


# Calculate Basic Statistics

Compute mean, median, and standard deviation for relevant columns in the dataset.

In [3]:
# Calculate basic statistics
mean_values = data.mean()
median_values = data.median()
std_values = data.std()

print("Mean Values:\n", mean_values)
print("Median Values:\n", median_values)
print("Standard Deviation:\n", std_values)

TypeError: Could not convert ['2024-01-01 00:00:002024-01-01 01:00:002024-01-01 02:00:002024-01-01 03:00:002024-01-01 04:00:002024-01-01 05:00:002024-01-01 06:00:002024-01-01 07:00:002024-01-01 08:00:002024-01-01 09:00:002024-01-01 10:00:002024-01-01 11:00:002024-01-01 12:00:002024-01-01 13:00:002024-01-01 14:00:002024-01-01 15:00:002024-01-01 16:00:002024-01-01 17:00:002024-01-01 18:00:002024-01-01 19:00:002024-01-01 20:00:002024-01-01 21:00:002024-01-01 22:00:002024-01-01 23:00:002024-01-02 00:00:002024-01-02 01:00:002024-01-02 02:00:002024-01-02 03:00:002024-01-02 04:00:002024-01-02 05:00:002024-01-02 06:00:002024-01-02 07:00:002024-01-02 08:00:002024-01-02 09:00:002024-01-02 10:00:002024-01-02 11:00:002024-01-02 12:00:002024-01-02 13:00:002024-01-02 14:00:002024-01-02 15:00:002024-01-02 16:00:002024-01-02 17:00:002024-01-02 18:00:002024-01-02 19:00:002024-01-02 20:00:002024-01-02 21:00:002024-01-02 22:00:002024-01-02 23:00:002024-01-03 00:00:002024-01-03 01:00:002024-01-03 02:00:00

# Identify Load Patterns Over Time

Analyze load patterns by aggregating data over time intervals (e.g., hourly, daily) and plotting time series graphs.

In [ ]:
import matplotlib.pyplot as plt

# Convert time column to datetime if available
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Aggregate data hourly
data['hour'] = data['timestamp'].dt.hour
hourly_load = data.groupby('hour').mean()

# Plot load patterns
plt.figure(figsize=(10, 6))
plt.plot(hourly_load.index, hourly_load['load'], marker='o')
plt.title('Hourly Load Patterns')
plt.xlabel('Hour of Day')
plt.ylabel('Average Load')
plt.grid()
plt.show()

# Calculate Station-wise Statistics

Compute the mean and standard deviation for each station (001, 002, 003) based on the provided columns.

In [ ]:
# Filter data for each station
stations = ['001', '002', '003']
station_stats = {}

for station in stations:
    station_data = data[data['station_id'] == station]
    station_stats[station] = {
        'mean': station_data.mean(),
        'std': station_data.std()
    }

# Display the results
for station, stats in station_stats.items():
    print(f"Station {station} Statistics:")
    print("Mean:\n", stats['mean'])
    print("Standard Deviation:\n", stats['std'])
    print("-" * 40)